In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score,accuracy_score,confusion_matrix, f1_score, precision_score, auc,roc_auc_score,roc_curve, precision_recall_curve
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.combine import SMOTEENN,SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('data_source/log2.csv')
data.head()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18


In [3]:
data['Action'].replace({'allow':0, 'deny':1, 'drop':2, 'reset-both':3},inplace=True)

data.tail(10)

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
65522,53314,64097,0,0,1,66,66,0,1,0,1,0
65523,55142,26467,0,0,1,66,66,0,1,0,1,0
65524,53848,36237,0,0,1,62,62,0,1,0,1,0
65525,65323,53,33275,53,0,356,118,238,2,30,1,1
65526,51710,43069,65147,43069,0,70,70,0,2,8,2,0
65527,63691,80,13237,80,0,314,192,122,6,15,4,2
65528,50964,80,13485,80,0,4680740,67312,4613428,4675,77,985,3690
65529,54871,445,0,0,2,70,70,0,1,0,1,0
65530,54870,445,0,0,2,70,70,0,1,0,1,0
65531,54867,445,0,0,2,70,70,0,1,0,1,0


Check magnitude of imbalance

In [4]:
data['Action'].value_counts()

0    37640
1    14987
2    12851
3       54
Name: Action, dtype: int64

Split into training and test datasets

In [5]:
# split the data into train and test set

X = data.iloc[:, data.columns != 'Action']
y = data.iloc[:, data.columns == 'Action']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0,stratify=data['Action'])

print(X_train.shape)
print(X_test.shape)

(45872, 11)
(19660, 11)


1. SMOTETomek

In [6]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(sampling_strategy='auto')
X_smt, y_smt = smt.fit_resample(X_train, y_train)

print(y_train.value_counts())
print(y_smt.value_counts())

knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_smt,y_smt)

train_yhat = knn.predict(X_smt)
train_f1 = f1_score(y_smt,train_yhat,average='macro')

test_yhat = knn.predict(X_test)
test_f1 = f1_score(y_test,test_yhat,average='macro')
print('Train Score\n',classification_report(y_smt,train_yhat))
print('Test Score\n',classification_report(y_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

from sklearn.model_selection import cross_val_score
print(cross_val_score(knn, X_smt, y_smt, cv=5))

Action
0         26348
1         10491
2          8995
3            38
dtype: int64
Action
2         26342
0         26329
3         26310
1         26287
dtype: int64


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26329
           1       1.00      1.00      1.00     26287
           2       1.00      1.00      1.00     26342
           3       1.00      1.00      1.00     26310

    accuracy                           1.00    105268
   macro avg       1.00      1.00      1.00    105268
weighted avg       1.00      1.00      1.00    105268

Test Score
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     11292
           1       0.99      0.97      0.98      4496
           2       1.00      1.00      1.00      3856
           3       0.09      0.75      0.15        16

    accuracy                           0.99     19660
   macro avg       0.77      0.93      0.78     19660
weighted avg       0.99      0.99      0.99     19660

AVG F1-Score Train: 0.9994108071685821
AVG F1-Score Test: 0.781558539789828


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

[0.9907856  0.9908331  0.99202052 0.99225764 0.99012017]


2. SMOTEENN

In [7]:
from imblearn.combine import SMOTEENN

sme = SMOTEENN(sampling_strategy='auto')
X_sme, y_sme = sme.fit_resample(X_train, y_train)

print(y_train.value_counts())
print(y_sme.value_counts())

knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_sme,y_sme)

train_yhat = knn.predict(X_sme)
train_f1 = f1_score(y_sme,train_yhat,average='macro')

test_yhat = knn.predict(X_test)
test_f1 = f1_score(y_test,test_yhat,average='macro')
print('Train Score\n',classification_report(y_sme,train_yhat))
print('Test Score\n',classification_report(y_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

from sklearn.model_selection import cross_val_score
print(cross_val_score(knn, X_sme, y_sme, cv=5))

Action
0         26348
1         10491
2          8995
3            38
dtype: int64
Action
2         26266
0         26041
3         25662
1         25367
dtype: int64


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26041
           1       1.00      1.00      1.00     25367
           2       1.00      1.00      1.00     26266
           3       1.00      1.00      1.00     25662

    accuracy                           1.00    103336
   macro avg       1.00      1.00      1.00    103336
weighted avg       1.00      1.00      1.00    103336

Test Score
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     11292
           1       0.98      0.95      0.96      4496
           2       0.99      1.00      1.00      3856
           3       0.05      0.75      0.09        16

    accuracy                           0.98     19660
   macro avg       0.76      0.92      0.76     19660
weighted avg       0.99      0.98      0.99     19660

AVG F1-Score Train: 1.0
AVG F1-Score Test: 0.7614968357964373


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

[0.99883878 0.99898389 0.9985968  0.99883873 0.99869357]


3. RandomOverSampler

In [8]:
ros = RandomOverSampler(sampling_strategy='not majority')
X_ros, y_ros = ros.fit_resample(X_train, y_train)

print(y_train.value_counts())
print(y_ros.value_counts())

knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_ros,y_ros)

train_yhat = knn.predict(X_ros)
train_f1 = f1_score(y_ros,train_yhat,average='macro')

test_yhat = knn.predict(X_test)
test_f1 = f1_score(y_test,test_yhat,average='macro')
print('Train Score\n',classification_report(y_ros,train_yhat))
print('Test Score\n',classification_report(y_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

from sklearn.model_selection import cross_val_score
print(cross_val_score(knn, X_ros, y_ros, cv=5))

Action
0         26348
1         10491
2          8995
3            38
dtype: int64
Action
0         26348
1         26348
2         26348
3         26348
dtype: int64


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26348
           1       1.00      1.00      1.00     26348
           2       1.00      1.00      1.00     26348
           3       1.00      1.00      1.00     26348

    accuracy                           1.00    105392
   macro avg       1.00      1.00      1.00    105392
weighted avg       1.00      1.00      1.00    105392

Test Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     11292
           1       0.98      0.99      0.99      4496
           2       1.00      0.99      1.00      3856
           3       0.67      0.25      0.36        16

    accuracy                           0.99     19660
   macro avg       0.91      0.81      0.84     19660
weighted avg       0.99      0.99      0.99     19660

AVG F1-Score Train: 0.9998576842828465
AVG F1-Score Test: 0.8358427692812443


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

[0.99705868 0.99786517 0.9976753  0.99862416 0.99829206]


4. SMOTE

In [9]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='not majority')
X_sm, y_sm = sm.fit_resample(X_train, y_train)

print(y_train.value_counts())
print(y_sm.value_counts())

knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_sm,y_sm)

train_yhat = knn.predict(X_sm)
train_f1 = f1_score(y_sm,train_yhat,average='macro')

test_yhat = knn.predict(X_test)
test_f1 = f1_score(y_test,test_yhat,average='macro')
print('Train Score\n',classification_report(y_sm,train_yhat))
print('Test Score\n',classification_report(y_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

from sklearn.model_selection import cross_val_score
print(cross_val_score(knn, X_sm, y_sm, cv=5))

Action
0         26348
1         10491
2          8995
3            38
dtype: int64
Action
0         26348
1         26348
2         26348
3         26348
dtype: int64


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26348
           1       1.00      1.00      1.00     26348
           2       1.00      1.00      1.00     26348
           3       1.00      1.00      1.00     26348

    accuracy                           1.00    105392
   macro avg       1.00      1.00      1.00    105392
weighted avg       1.00      1.00      1.00    105392

Test Score
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     11292
           1       0.99      0.97      0.98      4496
           2       1.00      1.00      1.00      3856
           3       0.10      0.75      0.17        16

    accuracy                           0.99     19660
   macro avg       0.77      0.93      0.79     19660
weighted avg       0.99      0.99      0.99     19660

AVG F1-Score Train: 0.9997343410253653
AVG F1-Score Test: 0.7857036176765935


d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

[0.98937331 0.99013236 0.99093842 0.99221938 0.99188728]


Finally, design to use combine sampling as SMOTEENN

In [10]:
# save the data

# save combine sampling dataset
X_sme.to_csv('data_source/X_train.csv',index=False)
y_sme.to_csv('data_source/y_train.csv',index=False)

# save test dataset
X_test.to_csv('data_source/X_test.csv',index=False)
y_test.to_csv('data_source/y_test.csv',index=False)